# Brazilian public e-commerce dataset analysis

## Project objectives:

- Clustering the costumers
- Predict future sales
- Delivery performance analysis
- Product quality
- Feature engineering

## Dataset context

This dataset was obtained though [Kaggle public datasets](https://www.kaggle.com/olistbr/brazilian-ecommerce) and it was originally provided by Olist, the largest department store in Brazilian marketplaces. Olist connects small businesses from all over Brazil to channels without hassle and with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners.

After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.

This is real commercial data, it has been anonymised, and references to the companies and partners in the review text have been replaced with the names of Game of Thrones great houses.

The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil.


### Data scheme
![](https://i.imgur.com/HRhd2Y0.png)


In [18]:
import pandas as pd

In [ ]:
# Files names:
olist_customers_dataset.csv
olist_geolocation_dataset.csv
olist_orders_dataset.csv
olist_order_items_dataset.csv
olist_order_payments_dataset.csv
olist_order_reviews_dataset.csv
olist_products_dataset.csv
olist_sellers_dataset.csv
product_category_name_translation.csv

In [20]:
# reading the files
raw_path = "D:\Ivan\Documentos\Documentos - Outros\Data Science\Github\Ivan_Portfolio\database\Brazilian e-commerce\"
olist_customer = pd.read_csv(raw_path + 'olist_customers_dataset.csv')
olist_geolocation = pd.read_csv(raw_path + 'olist_geolocation_dataset.csv')
olist_orders = pd.read_csv(raw_path + 'olist_orders_dataset.csv')
olist_order_items = pd.read_csv(raw_path + 'olist_order_items_dataset.csv')
olist_order_payments = pd.read_csv(raw_path + 'olist_order_payments_dataset.csv')
olist_order_reviews = pd.read_csv(raw_path + 'olist_order_reviews_dataset.csv')
olist_products = pd.read_csv(raw_path + 'olist_products_dataset.csv')
olist_sellers = pd.read_csv(raw_path + 'olist_sellers_dataset.csv')

SyntaxError: EOL while scanning string literal (<ipython-input-20-5c6cf3673e86>, line 2)